# CPSC 393 Machine Learning - Homework 4
By Matthew Favela

In [8]:
import keras as kb

import numpy as np
import string
from random import randint
from pickle import load
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

/Users/matt/Documents/School/CPSC393/favela_matt_hw4_cpsc_393/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def load_doc(filename):
 file = open(filename, 'r')
 text = file.read()
 file.close()
 return text

def clean_doc(doc):
 doc = doc.replace('--', ' ')
 tokens = doc.split()
 table = str.maketrans('', '', string.punctuation)
 tokens = [w.translate(table) for w in tokens]
 tokens = [word for word in tokens if word.isalpha()]
 tokens = [word.lower() for word in tokens]
 return tokens

def save_doc(lines, filename):
 data = '\n'.join(lines)
 file = open(filename, 'w')
 file.write(data)
 file.close()

in_filename = "alices_adventure_in_wonderland.txt"
doc = load_doc(in_filename)
print(doc[:200])

tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
 seq = tokens[i-length:i]
 line = ' '.join(seq)
 sequences.append(line)
print('Total Sequences: %d' % len(sequences))

out_filename = '{}_sequence.txt'.format(in_filename)
save_doc(sequences, out_filename)

in_filename = '{}_sequence.txt'.format(in_filename)
doc = load_doc(in_filename)
lines = doc.split('\n')

The Project Gutenberg eBook of Alice's Adventures in Wonderland
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no r
['the', 'project', 'gutenberg', 'ebook', 'of', 'alices', 'adventures', 'in', 'wonderland', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 'reuse', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'wwwgutenbergorg', 'if', 'you', 'are', 'not', 'located', 'in', 'the', 'united', 'states', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'ebook', 'title', 'alices', 'adventu

In [9]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
vocab_size = len(tokenizer.word_index) + 1
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = kb.utils.to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 50, input_shape=(seq_length,)))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

model.save('model.tf')

/Users/matt/Documents/School/CPSC393/favela_matt_hw4_cpsc_393/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 50, 50)         │       142,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        60,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2856)           │       288,456 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 491,656 (1.88 MB)

 Trainable params: 491,656 (1.88 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100
643/825 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.0635 - loss: 6.5239

KeyboardInterrupt: 

In [7]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
 result = list()
 in_text = seed_text
 for _ in range(n_words):
  encoded = tokenizer.texts_to_sequences([in_text])[0]
  encoded = kb.preprocessing.sequence.pad_sequences([encoded], maxlen=seq_length, truncating='pre')
  yhat = np.argmax(model.predict(encoded, verbose = 0), axis=-1)
  out_word = ''
  for word, index in tokenizer.word_index.items():
    if index == yhat:
      out_word = word
      break
  in_text += ' ' + out_word
  result.append(out_word)
 return ' '.join(result)

doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

seed_text = lines[randint(0,len(lines))]
print("SEED TEXT:", seed_text)

generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print("GENDERATED TEXT:", generated + '\n\n')

SEED TEXT: with another dig of her sharp little chin a right to said alice sharply for she was beginning to feel a little worried about as much said the duchess pigs have to fly and the but here to great surprise the voice died away even in the middle of her favourite


NameError: name 'model' is not defined